# Generate synonyms in the word embedding space

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastaug.util import DownloadUtil, EmbeddingNbrUtil
import torch
import numpy as np
import json

In [3]:
import pandas
embed_path = DownloadUtil.download_counter_fitting_if_not_exists()
df = pandas.read_csv(embed_path, sep=" ", header=None, na_values=None)
words = list(map(str, df.values[:, 0]))
vecs = torch.tensor(df.values[:, 1:].astype(np.float32))
embed_util = EmbeddingNbrUtil(
    vecs.cuda(),
    word2idx={words[i]: i for i in range(len(words))},
    idx2word={i: words[i] for i in range(len(words))},
)

File exists...


In [46]:
embed_util.find_neighbours('good', measure='cos', topk=17, dist=0.25, return_words=True)

['good',
 'great',
 'better',
 'very',
 'nice',
 'really',
 'excellent',
 'decent',
 'well',
 'but']

In [47]:
cands = {}
word2idx={words[i]: i for i in range(len(words))}
from tqdm import tqdm
for word in tqdm(word2idx):
    if isinstance(word, str):
        nbrs = embed_util.find_neighbours(word, measure='cos', topk=17, dist=0.25, return_words=True)
        nbrs.remove(word)
        cands[word] = nbrs

100%|██████████| 76854/76854 [03:43<00:00, 343.96it/s]


In [3]:
from fastaug.aug_ops.word import WordEmbedSub, WordNetSub, WordDictSub
gen = WordDictSub(0.1, cands)
emb = WordEmbedSub(0.1)
# emb.cands = cands
net = WordNetSub(0.1)
words = "A wiki is a hypertext publication collaboratively edited and managed by its own audience directly using a web browser.".split(" ")
for word in words:
    print(word)
    if gen.has_cands(word):
        print("gen-->", gen.get_cands(word))
    if emb.has_cands(word):
        print("emb-->", emb.get_cands(word))
    if net.has_cands(word):
        print("net-->", net.get_cands(word))
    print()

NameError: name 'cands' is not defined

In [ ]:
json.dump(cands, open("fastaug/resources/embed_top_16_dist_dot25.json", "w"))